In [38]:
import pandas as pd
school_df = pd.read_csv('../data/social_explorer_raw/School_Digger_2018.csv')
school_df = school_df.drop(index=0)
school_df = school_df.reset_index(drop=True)
print(school_df.shape)
school_df.head()

(19349, 30)


,Qualifying Name,Name of Area,FIPS,Nation,State,LEA,School,SchoolDigger Number of Stars Elementary,SchoolDigger Number of Stars High,SchoolDigger Number of Stars Middle,...,Number of Hispanic students (no value means redacted),Number of American Indian/Alaska Native students (no value means redacted),Number of Hawaiian Native/Pacific Islander students (no value means redacted),Number of students with Two or More Races (no value means redacted),Total Student membership/Full-Time-Equvalent teachers (no value means redacted),Number of Charter schools,Number of Students Receiving Free or DisNumbered Lunch (no value means redacted),Number of Private Schools,Number of Magnet schools,Number of Virtual schools
0,"Alabama Youth Services, Alabama",Alabama Youth Services,0100002,00,01,00002,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Albertville City, Alabama",Albertville City,0100005,00,01,00005,NaN,2,2,3,...,2709,14,3,150,19.3666666666667,0,3877,0,0,0
2,"Marshall County, Alabama",Marshall County,0100006,00,01,00006,NaN,1.75,2.75,2,...,1281,31,1,74,19.7923076923077,0,3977,0,0,0
3,"Hoover City, Alabama",Hoover City,0100007,00,01,00007,NaN,4.22222222222222,5,5,...,1095,10,10,686,14.75,0,2954,0,0,0
4,"Madison City, Alabama",Madison City,0100008,00,01,00008,NaN,5,5,5,...,511,80,39,529,18.8090909090909,0,2096,0,0,0


In [45]:
crime_df = pd.read_csv('../data/social_explorer_raw/FBI_Crime_Data_2019.csv')
crime_df = crime_df.drop(index=0)
crime_df = crime_df.reset_index(drop=True)
print(crime_df.shape)
print(crime_df.isna().any(axis=1).sum())
crime_df.head(1000)

(3143, 27)
3143


,Qualifying Name,Name of Area,FIPS,Nation,State,County,Total Population (2019 est.),Total Violent and Property Crimes,Total Violent and Property Crimes: Violent Crimes,Total Violent and Property Crimes: Property Crimes,...,Total Property Crimes: Burglaries,Total Property Crimes: Larcenies,Total Property Crimes: Motor Vehicle Thefts,Arsons,Total Number of Hate Crime Reported:,"Total Number of Hate Crimes with an Anti-Race, Anti-Ethnicity, Anti-Ancestry Bias Reported:",Total Number of Hate Crimes with an Anti-Religion Bias Reported:,Total Number of Hate Crimes with an Anti-Sexual Orientation Bias Reported:,Total Number of Hate Crimes with an Anti-Disability Bias Reported:,Total Number of Hate Crimes with an Anti-Gender and Anti-Gender Identity Bias Reported:
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,55869,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,223234,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,24686,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,22394,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Blount County, Alabama",Blount County,01009,00,01,009,57826,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"Allen County, Kentucky",Allen County,21003,00,21,003,21315,94,8,86,...,38,37,11,2,NaN,NaN,NaN,NaN,NaN,NaN
996,"Anderson County, Kentucky",Anderson County,21005,00,21,005,22747,17,2,15,...,4,7,4,0,NaN,NaN,NaN,NaN,NaN,NaN
997,"Ballard County, Kentucky",Ballard County,21007,00,21,007,7888,71,9,62,...,21,32,9,0,NaN,NaN,NaN,NaN,NaN,NaN
998,"Barren County, Kentucky",Barren County,21009,00,21,009,44249,114,30,84,...,22,43,19,0,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
health_df = pd.read_csv('../data/social_explorer_raw/Health_data_2023_Release.csv')
health_df = health_df.drop(index=0)
health_df = health_df.reset_index(drop=True)
print(health_df.shape)
health_df.head()

(3221, 51)


,Qualifying Name,Name of Area,FIPS,Nation,State,County,Physically Unhealthy Days per Month (Persons 18 Years and Over),Mentally Unhealthy Days per Month (Persons 18 Years and Over),Percent of Adults That Report Fair or Poor Health (Persons 18 Years and Over),Percent of Low Birthweight Births (<2.5kg),...,Food Environment Index,Air Pollution Particulate Matter Average Daily Pm2.5,Presence of Drinking Water Violations,Percentage of Households with Severe Housing Problems,Percentage of Households with High Housing Costs,Percentage of Households with Overcrowding,Percentage of Households with Lack of Kitchen or Plumbing Facilities,Percent of Driving Alone to Work,Long Commute Driving Alone Workers Who Drive Alone,Percent of Long Commute Driving Alone Workers Who Drive Alone
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,3.4322111963,4.7973512632,16.9,9.7381519152,...,6.6,10,No,14.442626782,12.635805385,1.12175742,1.6358962374,86.401088518,26458,41.6
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,3.2761769521,4.7537497106,14.9,8.3857311835,...,7.5,7.6,Yes,12.238971951,10.561056106,1.2912393426,0.642530582,81.40185095,102650,37.6
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,4.6054318631,4.9548545646,27.5,11.914672216,...,5.8,9.4,Yes,16.21187801,13.464345873,3.8523274478,0.5885500268,83.122562345,8461,36.5
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,4.0121820779,5.364779234,21.6,10.033076075,...,7.4,9.8,No,11.320754717,7.9853479853,1.1611030479,2.2496371553,86.965223918,7994,55.1
4,"Blount County, Alabama",Blount County,01009,00,01,009,3.8660480758,5.377579831,18.4,7.8598889364,...,7.8,9.6,No,9.8824658191,7.3806006893,1.7989925642,1.2712880787,86.420269253,23918,59.5


In [24]:
demographic_df = pd.read_csv('../data/social_explorer_raw/Census2020_PL94_Redistricting_Data.csv')
demographic_df = demographic_df.drop(index=0)
demographic_df = demographic_df.reset_index(drop=True)
print(demographic_df.shape)
demographic_df.head()

(3221, 97)


,Qualifying Name,Name of Area,FIPS,Nation,State,County,County Subdivision,Census Tract,Block Group,Place,...,Hispanic 2010,Population Change.2,Non-Hispanic Change,Hispanic Change,Population Change Percent.1,Non-Hispanic Change Percent,Hispanic Change Percent,Population Change Percent Points,Non-Hispanic Change Percent Points,Hispanic Change Percentage Points
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,NaN,NaN,NaN,NaN,...,1310,4234,3427,807,7.7586996756519,6.43435158934305,61.6030534351145,7.54951656745106E-15,-1.19949159807662,1.19949159807663
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,NaN,NaN,NaN,NaN,...,7992,49502,44808,4694,27.1593558829177,25.711383863249,58.7337337337337,-1.99840144432528E-15,-1.08877634772734,1.08877634772734
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,NaN,NaN,NaN,NaN,...,1387,-2234,-2357,123,-8.13635866992024,-9.04104334484081,8.86806056236482,-3.44169137633799E-15,-0.935064404518871,0.935064404518867
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,NaN,NaN,NaN,NaN,...,406,-622,-956,334,-2.71437922758019,-4.24719001288374,82.2660098522168,6.21724893790088E-15,-1.547662403732,1.547662403732
4,"Blount County, Alabama",Blount County,01009,00,01,009,NaN,NaN,NaN,NaN,...,4626,1812,667,1145,3.1610899829036,1.26575072111735,24.7514051015997,2.66453525910038E-15,-1.68899106671437,1.68899106671438


In [25]:
housing_df = pd.read_csv('../data/social_explorer_raw/Income_limits_and_fair_market_rent_2020.csv')
housing_df = housing_df.drop(index=0)
housing_df = housing_df.reset_index(drop=True)
print(housing_df.shape)
housing_df.head()

(4766, 41)


,Qualifying Name,Name of Area,FIPS,Nation,Metropolitan Statistical Area,County/County Subdivision,Total Population 2017,Median Family Income (5-year ACS),Low Income Limits (Up to 80% of Local Area Median Income) for One Person Families,Low Income Limits (Up to 80% of Local Area Median Income) for Two Person Families,...,Extremely Low Income Limits (Up to 30% of Local Area Median Income) for Eight Person Families,Fair Market Rent for Efficiency,Fair Market Rent for One Bedroom,Fair Market Rent for Two Bedrooms,Fair Market Rent for Three Bedrooms,Fair Market Rent for Four Bedrooms,FY 2020 Fair Market Rent (Two-Bedrooms),FY 2019 Fair Market Rent (Two-Bedrooms),Fair Market Rent (Two-Bedrooms) Percent Change between 2019 and 2020,Fair Market Rent (Two-Bedrooms) Dollar Change between 2019 and 2020
0,"Autauga County, Alabama",Autauga County,0100199999,00,NaN,00199999,55035,65700,36800,42050,...,43400,583,702,830,1047,1425,830,825,0.606061,5
1,"Baldwin County, Alabama",Baldwin County,0100399999,00,NaN,00399999,203360,81000,45400,51850,...,44120,744,749,916,1251,1566,916,888,3.15315,28
2,"Barbour County, Alabama",Barbour County,0100599999,00,NaN,00599999,26200,43800,30050,34350,...,35400,477,481,633,789,925,633,666,-4.955,-33
3,"Bibb County, Alabama",Bibb County,0100799999,00,NaN,00799999,22580,73100,40950,46800,...,44120,804,861,986,1291,1425,986,873,12.9438717,113
4,"Blount County, Alabama",Blount County,0100999999,00,NaN,00999999,57665,73100,40950,46800,...,44120,804,861,986,1291,1425,986,873,12.9438717,113


In [26]:
#economic indicator
unemployment_df = pd.read_csv('../data/social_explorer_raw/Unemployment_Statistics_2021.csv')
unemployment_df = unemployment_df.drop(index=0)
unemployment_df = unemployment_df.reset_index(drop=True)
print(unemployment_df.shape)
unemployment_df.head()

(3219, 39)


,Qualifying Name,Name of Area,FIPS,Nation,State,County,Metropolitan Statistical Area,Labor Force as of July 2021,Employed as of July 2021,Unemployed as of July 2021,...,Unemployed as of June 2021,Unemployment rate % as of June 2021,Labor Force as of July 2021.1,Employed as of July 2021.1,Unemployed as of July 2021.1,Unemployment rate % as of July 2021.1,Labor Force,Employed.1,Unemployed,Unemployment rate %.1
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,NaN,25727,25009,718,...,791,3.1,25727,25009,718,2.8,1.06061201241309,1.39057812373307,-9.22882427307206,-9.67741935483872
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,NaN,102740,99981,2759,...,3042,3,102740,99981,2759,2.7,1.69357313246692,2.03496382173145,-9.30309007232084,-9.99999999999999
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,NaN,7097,6651,446,...,459,6.6,7097,6651,446,6.3,1.51623515949077,1.82180036742192,-2.8322440087146,-4.54545454545454
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,NaN,8675,8381,294,...,321,3.8,8675,8381,294,3.4,1.42640009353443,1.81000971817298,-8.41121495327103,-10.5263157894737
4,"Blount County, Alabama",Blount County,01009,00,01,009,NaN,25212,24647,565,...,627,2.5,25212,24647,565,2.2,1.78852598005571,2.09179024107365,-9.88835725677831,-12


In [42]:

merged_df = pd.merge(crime_df, school_df, on='Qualifying Name', how='left')

print(merged_df.shape)
merged_df.head()

(3143, 56)


,Qualifying Name,Name of Area_x,FIPS_x,Nation_x,State_x,County,Total Population (2019 est.),Total Violent and Property Crimes,Total Violent and Property Crimes: Violent Crimes,Total Violent and Property Crimes: Property Crimes,...,Number of Hispanic students (no value means redacted),Number of American Indian/Alaska Native students (no value means redacted),Number of Hawaiian Native/Pacific Islander students (no value means redacted),Number of students with Two or More Races (no value means redacted),Total Student membership/Full-Time-Equvalent teachers (no value means redacted),Number of Charter schools,Number of Students Receiving Free or DisNumbered Lunch (no value means redacted),Number of Private Schools,Number of Magnet schools,Number of Virtual schools
0,"Autauga County, Alabama",Autauga County,01001,00,01,001,55869,NaN,NaN,NaN,...,292,28,11,203,18.4833333333333,0,4014,0,0,0
1,"Baldwin County, Alabama",Baldwin County,01003,00,01,003,223234,NaN,NaN,NaN,...,2894,99,23,1890,19.4404761904762,0,15410,0,0,1
2,"Barbour County, Alabama",Barbour County,01005,00,01,005,24686,NaN,NaN,NaN,...,120,3,2,2,18.1666666666667,0,772,0,0,0
3,"Bibb County, Alabama",Bibb County,01007,00,01,007,22394,NaN,NaN,NaN,...,111,4,0,33,17.6125,0,2002,0,0,0
4,"Blount County, Alabama",Blount County,01009,00,01,009,57826,NaN,NaN,NaN,...,1171,10,4,41,17.9142857142857,0,4229,0,0,0
